### Proteomic data

In [1]:
import pandas as pd
df = pd.read_csv('/home/biodata/aman/data/data_raw/paxdb_human_data/9606-WHOLE_ORGANISM-integrated.txt', sep="\t")
df

,#gene_name,string_external_id,abundance
0,TMSB4X,9606.ENSP00000370010,8.296400e+04
1,APOA2,9606.ENSP00000356969,3.136100e+04
2,RBP4,9606.ENSP00000360522,2.444300e+04
3,ORM1,9606.ENSP00000259396,2.283500e+04
4,APOA1,9606.ENSP00000236850,2.072300e+04
...,...,...,...
19478,OR4N5,9606.ENSP00000493307,5.000000e-07
19479,hCG_1796489,9606.ENSP00000489727,4.000000e-07
19480,OR5M8,9606.ENSP00000323354,4.000000e-07
19481,C12orf42,9606.ENSP00000367353,3.000000e-07


# Should I do bootstrapping here? to get std and cv??

In [2]:
import geckopy
model = geckopy.io.read_sbml_ec_model('/home/biodata/aman/data/ecHumanGEM.xml', hardcoded_rev_reactions=False)  # how is this made actually? principle? # parameter for faster loading
print('Human Model is imported ... ') # 8+ mins # why ec model taking more time than the normal one?ODD

Human Model is imported ... 


In [3]:
# import pickle
# # with open("ecHumanGEM.pkl", "wb") as f:
# #     pickle.dump(model, f)
# model = pickle.load(open("/home/biodata/aman/data/ecHumanGEM.pkl", "rb")) ## to load
# import geckopy
# model = geckopy.io.read_sbml_ec_model(model)
# print('Human Model is imported ... ')

In [4]:
model.proteins

[<Protein prot_Q8IWW8[c] at 0x72df99937160>,
 <Protein prot_P28332[c] at 0x72df99937190>,
 <Protein prot_Q8N4Q0[c] at 0x72df99937fd0>,
 <Protein prot_P07327[c] at 0x72df99937f40>,
 <Protein prot_P40394[c] at 0x72df998c71f0>,
 <Protein prot_P00325[c] at 0x72df998c7280>,
 <Protein prot_P11766[c] at 0x72df998c7310>,
 <Protein prot_P08319[c] at 0x72df998c73a0>,
 <Protein prot_P00326[c] at 0x72df998c7430>,
 <Protein prot_P14550[c] at 0x72df998c74c0>,
 <Protein prot_Q9NR19[c] at 0x72df998c7550>,
 <Protein prot_Q9H6R3[c] at 0x72df998c7670>,
 <Protein prot_Q9NUB1[c] at 0x72df998c76a0>,
 <Protein prot_O00330[c] at 0x72df998c7070>,
 <Protein prot_P08559[c] at 0x72df998c77c0>,
 <Protein prot_P09622[c] at 0x72df998c7850>,
 <Protein prot_P10515[c] at 0x72df998c78e0>,
 <Protein prot_P11177[c] at 0x72df998c7970>,
 <Protein prot_P29803[c] at 0x72df998c7a00>,
 <Protein prot_P07195[c] at 0x72df998c7b20>,
 <Protein prot_P00338[c] at 0x72df998c7b50>,
 <Protein prot_Q8IX04[c] at 0x72df998c7730>,
 <Protein 

In [6]:
import requests, time, pandas as pd

df = pd.read_csv('/home/biodata/aman/data/data_raw/paxdb_human_data/9606-WHOLE_ORGANISM-integrated.txt', sep='\t', comment="#", header=0)
df = df[:19000]

df["ENSP"] = df["string_external_id"].str.replace("9606.", "", regex=False)

ensps = df["ENSP"].drop_duplicates().tolist()

all_results = []

batch = 25   # because UniProt seems to output only 25

for i in range(0, len(ensps), batch):
    subset = ensps[i:i+batch]

    job = requests.post(
        "https://rest.uniprot.org/idmapping/run",
        data={
            "from": "Ensembl_Protein",
            "to": "UniProtKB",
            "ids": subset
        }
    ).json()

    job_id = job["jobId"]

    time.sleep(3)

    # Get results
    result = requests.get(
        f"https://rest.uniprot.org/idmapping/results/{job_id}"
    ).json()

    all_results.extend(result.get("results", []))

map_df = pd.DataFrame(all_results)
map_df = map_df.rename(columns={"from": "ENSP", "to": "UniProt"})

merged = df.merge(map_df, on="ENSP", how="left")

merged["protein_gecko_id"] = merged["UniProt"].apply(lambda x: f"prot_{x}[c]" if pd.notna(x) else None)

print(merged.head(20))
print("Total rows mapped:", merged["UniProt"].notna().sum())

ParserError: Error tokenizing data. C error: Expected 1 fields in line 11, saw 3


In [19]:
prot = merged[["UniProt", "abundance"]].copy()
prot = prot.dropna(subset=["UniProt"])
prot = prot.rename(columns={"abundance": "copies_per_cell"})

prot["protein_gecko_id"] = prot["UniProt"].apply(lambda x: f"prot_{x}[c]")

prot["stdev"] = 0

len(prot)

195

In [20]:
prot

,UniProt,copies_per_cell,protein_gecko_id,stdev
0,P62328,82964.0,prot_P62328[c],0
1,P02652,31361.0,prot_P02652[c],0
2,P02753,24443.0,prot_P02753[c],0
3,P02763,22835.0,prot_P02763[c],0
4,P02647,20723.0,prot_P02647[c],0
...,...,...,...,...
195,P68032,786.0,prot_P68032[c],0
196,P31949,775.0,prot_P31949[c],0
197,P63267,768.0,prot_P63267[c],0
198,P62736,763.0,prot_P62736[c],0


In [21]:
# prot

ec_model_exp = geckopy.experimental.from_copy_number(
    model,
    index=prot["protein_gecko_id"],
    cell_copies = prot["copies_per_cell"],
    stdev = prot["stdev"],
    vol=2.3e-12, dens=1.05, water=0.7
) # pipe output to additional log.f?

protein prot_P62328[c] from measurements was not found in the model
protein prot_P02763[c] from measurements was not found in the model
protein prot_P02768[c] from measurements was not found in the model
protein prot_P02766[c] from measurements was not found in the model
protein prot_P02655[c] from measurements was not found in the model
protein prot_P02654[c] from measurements was not found in the model
protein prot_P02790[c] from measurements was not found in the model
protein prot_P02656[c] from measurements was not found in the model
protein prot_C9JV77[c] from measurements was not found in the model
protein prot_P19652[c] from measurements was not found in the model
protein prot_P01009[c] from measurements was not found in the model
protein prot_P00738[c] from measurements was not found in the model
protein prot_P60709[c] from measurements was not found in the model
protein prot_P63261[c] from measurements was not found in the model
protein prot_P01008[c] from measurements was not

In [22]:
model.proteins

[<Protein prot_Q8IWW8[c] at 0x703222e880a0>,
 <Protein prot_P28332[c] at 0x703222e88040>,
 <Protein prot_Q8N4Q0[c] at 0x703222e88c70>,
 <Protein prot_P07327[c] at 0x703222e88d30>,
 <Protein prot_P40394[c] at 0x703222e88dc0>,
 <Protein prot_P00325[c] at 0x703222e88e50>,
 <Protein prot_P11766[c] at 0x703222e88ee0>,
 <Protein prot_P08319[c] at 0x703222e88f70>,
 <Protein prot_P00326[c] at 0x703222e88fa0>,
 <Protein prot_P14550[c] at 0x703222e88fd0>,
 <Protein prot_Q9NR19[c] at 0x703222e97160>,
 <Protein prot_Q9H6R3[c] at 0x703222e97280>,
 <Protein prot_Q9NUB1[c] at 0x703222e972b0>,
 <Protein prot_O00330[c] at 0x703222e97040>,
 <Protein prot_P08559[c] at 0x703222e973d0>,
 <Protein prot_P09622[c] at 0x703222e97460>,
 <Protein prot_P10515[c] at 0x703222e974f0>,
 <Protein prot_P11177[c] at 0x703222e97580>,
 <Protein prot_P29803[c] at 0x703222e97610>,
 <Protein prot_P07195[c] at 0x703222e97730>,
 <Protein prot_P00338[c] at 0x703222e97760>,
 <Protein prot_Q8IX04[c] at 0x703222e97340>,
 <Protein 

In [23]:
ec_model_exp

Name,M_ecHumanGEM_v1__46__5__46__0
Memory address,7031f079b610
Number of metabolites,12220
Number of reactions,43084
Number of genes,3626
Number of groups,0
Objective expression,1.0*biomass_human - 1.0*biomass_human_reverse_fb2f2
Compartments,"Cytosol, Extracellular, Lysosome, Endoplasmic reticulum, Mitochondria, Peroxisome, Golgi apparatus, Nucleus, Inner mitochondria"


In [24]:
ec_model_exp.slim_optimize()

187.35362997658075